In [ ]:
%load_ext autoreload
%autoreload 2

###Bao cam - 800 0.95 R101_FPN_3x

In [ ]:
from infer import *
import notebook_utils as nutils
import cv2
import os
import time
import matplotlib.pyplot as plt
from sort import *
from tqdm import tqdm

classes = ['BAO CAM']
# predictor = get_predictor("baocam_faster_rcnn_R_101_FPN_3x.yaml.pth", classes, 800 , 0.5)
predictor = get_predictor("baocam_faster_rcnn_R_101_FPN_3x.yaml.pth", classes, 800 , 0.95)

file = 'BAOCAM/6/7.JPG'
bboxes, classes, scores, v = infer('baocam', predictor, file, True, 0.5)

nutils.gdrive_down("1f6-Focz7eiRxia38H8nSxvghd5eAJxne", "CPL_FGAGro_38_63-20200917-070419.mp4")

In [ ]:
mp4_files = nutils.list_files("data", "CAM 1-*.mp4")
path = mp4_files[0]

In [ ]:
from pathlib import Path

In [ ]:
mp4_files = [Path("CPL_FGAGro_38_63-20200917-070419.mp4")]
path = mp4_files[0]

In [ ]:
for path in mp4_files:
    out = cv2.VideoWriter("temp.mp4",cv2.VideoWriter_fourcc(*'MP4V'), 15, (640, 360)) #(640, 360) (285, 360)
    
    vc = cv2.VideoCapture(str(path))
    time_per_frame = (1/vc.get(cv2.CAP_PROP_FPS)) * 1000
    #start_frame = 10800#12600 #10800
    vc.set(1,0);

    rval = True

    frame_id = 0
    max_frame = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
#     max_frame = 300

    counter = 0

    pbar = tqdm(total=max_frame, position=0, leave=True)
    
    mot_tracker = Sort(max_age=0, min_hits= 0, iou_threshold=0.1)
    mot_tracker.reset_count()
    while rval:
        rval , frame = vc.read()

        if rval:
            start = time.time()

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             image = image[..., 350:920, :].copy()
            
            bboxes, classes, scores, v = infer('baocam', predictor, image, True, 0.5)
#             print(scores)
#             ret, bboxes = process_stick_two_object(bboxes, 550)
            bboxes_with_scores = np.hstack((bboxes, scores.reshape(bboxes.shape[0],1)))
#             print(bboxes_with_scores)
            track_bbs_ids = mot_tracker.update(bboxes_with_scores)

            total_time = (time.time() - start) * 1000

#             v.draw_text("\n".join(["-".join(bbox + [str(round(scores[index], 3))]) for index, bbox in enumerate(bboxes.astype(np.int32).astype(np.str).tolist())]), 
#                         (200, 100), font_size= 30, color='g')


            for bbox in bboxes:
                v.draw_box(bbox, alpha = 1)

            for bbox in track_bbs_ids:
                counter = bbox[-1]
                v.draw_text(str(int(counter)), (int( (bbox[0] + bbox[2]) /2 ) , int((bbox[1] + bbox[3]) /2 )), font_size= 70, color='g')


            v.draw_text(str(int(counter)) + ("*" if False else ""), (100 , 100), font_size= 70, color='g')

            predicted_image = v.output.get_image()

            skip_frame = int(np.ceil(total_time / time_per_frame))
#             print(skip_frame)
#             skip_frame = 3
#             print(skip_frame)
            out.write(predicted_image)

            frame_id += 1
            pbar.update(1)
            
            for i in range(skip_frame+1):
                out.write(predicted_image)
                vc.read()
                pbar.update(1)
                frame_id += 1

            if frame_id > (max_frame - 0):
                break

    out.release()
    vc.release()

    new_file = "{}_export.mp4".format(path.stem).replace(" ", "") 
    cmd = "ffmpeg -i temp.mp4 -vcodec libx264 {} -y".format(new_file)
    print(cmd)
    os.system(cmd)

In [ ]:
from IPython.display import Video

Video(new_file, width = 200)


In [ ]:
out.release()
vc.release()

In [ ]:
v.draw_box(bboxes[0])

out = v.output.get_image()

nutils.imshow(out, 2)

In [ ]:
vc = cv2.VideoCapture(str(path))

start_frame = 12640
vc.set(1,start_frame);

mot_tracker = Sort(min_hits= 0) 
mot_tracker.reset_count()
counter = 0

In [ ]:
def process_stick_two_object(bboxes, thresh):
    a = bboxes.copy()
    
    max_height_to_split = thresh

    long_indices = np.where(a[:, 3] - a[:, 1] > max_height_to_split)[0]

    split_bboxes = []
    ret = False
    if len(long_indices) >0:
        print('There stick bboxes')
        ret = True
        for i in range(len(a)):
            if len(np.where(long_indices == i)[0]) > 0:
                bbox = a[i]
                center_height = bbox[3] - bbox[1]
                center_height *= 0.7

                split_bboxes.append([bbox[0], bbox[1], bbox[2], center_height])
                split_bboxes.append([bbox[0], center_height, bbox[2], bbox[3]])
            else:
                split_bboxes.append(a[i])
        
        a = np.array(split_bboxes)
    
    return ret, a
    

In [ ]:
vc = cv2.VideoCapture(str(path))
vc.set(1, 4000);
mot_tracker = Sort(max_age = 0, min_hits= 0)
mot_tracker.reset_count()

In [ ]:
counter = 0
time_per_frame = (1/30) * 1000

In [ ]:
classes = ['BAO CAM']
predictor = get_predictor("baocam.pth", classes, 400 , 0.9)


In [ ]:
rval , frame = vc.read()
start = time.time()

image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# image = image[..., 350:920, :].copy()

bboxes, classes, scores, v = infer('baocam', predictor, image, True, 0.5)

print(bboxes, scores)
# ret, bboxes = process_stick_two_object(bboxes, 530)
# print(bboxes)

track_bbs_ids = mot_tracker.update(bboxes)


for bbox in bboxes:
    v.draw_box(bbox, 1)
    
    
for bbox in track_bbs_ids:
    v.draw_box(bbox[:4], edge_color='r')
    v.draw_text(str(int(bbox[-1])), (int( (bbox[0] + bbox[2]) /2 ) , int((bbox[1] + bbox[3]) /2 )), font_size= 40, color='g')

try:
    counter = max(track_bbs_ids[..., -1])
except:
    pass
v.draw_text(str(int(counter)), (50 , 100), font_size= 50, color='g')

predicted_image = v.output.get_image()

total_time = (time.time() - start) * 1000

skip_frame = int(np.ceil(total_time / time_per_frame))


for i in range(skip_frame):
    vc.read()
    
nutils.imshow(predicted_image, 12)

In [ ]:
process_stick_two_object(a)

In [ ]:
a = np.array([[475.5481 , 107.80378, 801.6652 , 705.3319 ]])

In [ ]:
max_height_to_split = 500

long_indices = np.where(a[:, 3] - a[:, 1] > max_height_to_split)[0]

split_bboxes = []

if len(long_indices) >0:
    for i in range(len(a)):
        if len(np.where(long_indices == i)[0]) > 0:
            bbox = a[i]
            center_height = bbox[3] - bbox[1]
            center_height *= 0.7
            
            split_bboxes.append([bbox[0], bbox[1], bbox[2], center_height])
            split_bboxes.append([bbox[0], center_height, bbox[2], bbox[3]])
        else:
            split_bboxes.append(a[i])
        print(i)
        print(split_bboxes)
            
        
a = np.array(split_bboxes)
    

In [ ]:
range(len(a))

In [ ]:
a